### Get the stt_name of all the pdfs files

In [11]:
import os

def extract_names_from_pdfs(folder_path):
    """
    Extracts a specified part of PDF file names in a given folder and maps them to
    the full file names in a dictionary.

    Parameters:
        folder_path (str): The path to the folder containing the PDF files.

    Returns:
        Dict[str, str]: A dictionary mapping extracted name parts to full file names.
    """
    # Get all file names in the folder
    all_files = os.listdir(folder_path)
    
    # Filter for PDF files
    pdf_files = [f for f in all_files if f.lower().endswith('.pdf')]
    
    # Extract the desired part of the file name and store in a dictionary
    # with the extracted name as the key and the full file name as the value
    extracted_names = {}
    for filename in pdf_files:
        if "_TRAN_" in filename:
            key = filename.split("_TRAN_")[0].lower()
        elif "_LOC_" in filename:
            key = filename.split("_LOC_")[0].lower()
        elif  "_LOCATIONS_" in filename :  
            key = filename.split("_LOCATIONS_")[0].lower() 
        else:
            continue
        extracted_names[key] = filename
    
    return extracted_names

### Read the pdf's using pdfplumber

In [12]:
# Read the  pdf

import pdfplumber

# settings of extract table 
table_settings = {
    "vertical_strategy": "lines",
    "horizontal_strategy": "lines",
    "snap_y_tolerance": 5,
    "intersection_x_tolerance": 15,
}

def extract_tables_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        all_tables = []
        for page in pdf.pages:
            tables = page.extract_tables(table_settings)
            for table in tables:
                all_tables.append(table)
        all_tables = [pd.DataFrame(table) for table in all_tables]
                
        all_tables = [x for x in all_tables if (len(x) > 20) & (x.shape[1] > 5 ) ]#& (len(x) < 31)
        return all_tables

### Extract the commune names from each df 

In [13]:
def find_first_occurrence(df, targets=["minimal", "minimum"]):
    """Find the row index of the first occurrence of target texts in a DataFrame."""
    
    for row_index, row in df.iterrows():
        for value in row:
            if isinstance(value, str) and any(target in value.lower() for target in targets):
                return row_index
    return None
    
def extract_commune_name_v21(df):
    """Revised function with adjusted logic for the new case."""
    
    name = str(df.iloc[0, 0])+str(df.iloc[0, 1])+str(df.iloc[1, 0]) + str(df.iloc[1, 1])
    # Remove the specified strings from the combined content
    for to_remove in ["Commune de", "Commune d'", "Communes d'","Nature de l’Immeuble", 
                      "Type d’Immeuble /(Caractéristiques)","Type d'Immeuble / (caractéristiques)",
                      'None',"Type d’Immeuble (Caractéristiques)","Nature de\nl’Immeuble","Commune",
                      "Type d’Immeuble(Caractéristiques)","Type d’Immeuble /(caracteristiq","s de",
                      "Type d’Immeuble /(Caractéristiq","Partie Ouest et Sud(2): Autres quartiers",'province',
                      'Type d’Immeuble (Caractéristiques)','NOUVELLE VILLE',"Type d’Immeuble / (Caractéristiques)",
                     "Type d’Immeuble(caracteristique )","commune de","commune","COMMUNE","(*)","(M'cil)","COMMUNE D",
                         "Type d’Immeuble (caracteristique )",'Partie "Est" (1) : Quartiers El-Barqui, Victor Hugo, Petit Lac, Bel Air, Saint-Eugène, Delmonte, Carteaux, Centre ville, Gambetta, Point du jour etFalaises.',
                             	]:
        name = name.replace(to_remove, '')
    name  = name.strip()
    
     
    if find_first_occurrence(df) == 3:
        return name, df
    else:
        df = df.drop(0).reset_index(drop=True)    
        return name, df

## Apartments part

In [14]:
"""def clean_table_v5(df):
    #Enhanced function to clean and process the table with improved commune name extraction.
    name, df =  extract_commune_name_v21(df)
    try:
        # Step 1: Commune name is already provided
        
        # Step 2: Adjusting header, dropping unnecessary columns
        df_cleaned = df.iloc[4:, :].reset_index(drop=True)
        
        # Step 3: Assign new column names
        new_column_names = [
            "type_1", "type_2", "residential_min", "residential_madef" ]"""
        
def clean_table_v5(df):
    """Enhanced function to clean and process the table with improved commune name extraction."""
    name, df =  extract_commune_name_v21(df)
    try:
        # Step 1: Commune name is already provided
        
        # Step 2: Adjusting header, dropping unnecessary columns
        df_cleaned = df.iloc[4:, :].reset_index(drop=True)
        
        # Step 3: Assign new column names
        new_column_names = [
            "type_1", "type_2", "residential_min", "residential_max", 
            "centre_min", "centre_max", "peripherique_min", "px", 
            "centre_min", "centre_max", "peripherique_min", "peripherique_max","eloigner_min", "eloigner_max"
        ]
        
        if len(df_cleaned.columns) != len(new_column_names):
            print(f"Warning: Skipping table {name} due to unexpected column count.")
            return None

        df_cleaned.columns = new_column_names
        
        # Step 4: Further cleaning
        df_cleaned['type_2'].iloc[3] = 'terrain_2_facade'
        df_cleaned['type_2'].iloc[4] = 'terrain_1_facade'
        
        # Step 5: Numeric Conversion
        numeric_columns = [
            "residential_min", "residential_max", 
            "centre_min", "centre_max", 
            "peripherique_min", "peripherique_max",
            "eloigner_min", "eloigner_max"
        ]
        
        for col in numeric_columns:
            df_cleaned[col] = df_cleaned[col].apply(
                    lambda x: float(x.replace(" ", "").replace("*", "").replace("-", "nan").replace("#REF!", "nan").replace('esavecraccordements','nan').replace('le','nan').replace('cordement','nan').replace('690\n552\n414\n276\n221\n124\n109', '690').replace('avecracnans', 'nan').replace('essansracnans', 'nan').replace('sansracnans', 'nan').replace("nts", "").replace("s", "").replace("t", "")) if isinstance(x, str) and x.strip() not in ["", "/"] else None
            )
        
        # Step 6: Filtering Rows
        df_cleaned = df_cleaned.iloc[[2, 13, 14,15,16,17,18], :]
        
        # Step 7: Setting 'type_2' as the index
        df_cleaned.set_index('type_2', inplace=True)

        # rename the indexies 
        new_index_names = ['Economique', 'Trerrain_2', 'Trerrain_1',]
        df_cleaned.index = new_index_names

        return name, df_cleaned
        
    except Exception as e:
        print(f'There is an error: {e}')
        return None

In [15]:
import pandas as pd
from pdfminer.pdfparser import PDFSyntaxError

pdfs_path = 'pdf/'   # put the location of the pdf's

all_data =  []

# get  the names of all the pdfs in dict format 
pdfs_in_folder = extract_names_from_pdfs(pdfs_path)

print(pdfs_in_folder)

for key, value in pdfs_in_folder.items():

    print(key)
    # READ THE PDFS 
    try:
        dfs = extract_tables_from_pdf(pdfs_path+value)
    except PDFSyntaxError:
        print(f"Warning: The file {value} seems to be an invalid PDF. Skipping...")
        continue
    
    #dfs = extract_tables_from_pdf(pdfs_path+value)

    # Create a list of cleaned dataframes with their respective names
    cleaned_dfs = [clean_table_v5(d) for d in dfs]
    

    for i, commune in enumerate(cleaned_dfs):
        try:
            # center
            centre_min  =commune[1].loc['Economique', 'centre_min'] 
            centre_max = commune[1].loc['Economique', 'centre_max']

                # perepheric
            peripherique_min  =commune[1].loc['Economique', 'peripherique_min'] 
            peripherique_max = commune[1].loc['Economique', 'peripherique_max']

                # calculate the mean_eloigner
            eloigner_min  =commune[1].loc['Economique', 'eloigner_min'] 
            eloigner_max = commune[1].loc['Economique', 'eloigner_max']

            

            all_data.append({
                    'stt_name': key,
                    'commune_name': commune[0],
                    'centre_min': centre_min,
                    'centre_max': centre_max,
                    'peripherique_min': peripherique_min,  # This will be either 'village' or 'city'
                    'peripherique_max': peripherique_max,
                    'far_min':eloigner_min,
                    'far_max':eloigner_max
                                })
        except Exception as e:
            print(f'The error is wilaya {key}, commune {i}')

# Display the first few entries in all_data_revised
impo_df = pd.DataFrame(all_data)    

{'ouargla': 'OUARGLA_TRAN_23.pdf', 'ain_defla': 'AIN_DEFLA_TRAN_23.pdf', 'saida': 'SAIDA_TRAN_23.pdf', 'ouled_djellal': 'OULED_DJELLAL_TRAN_23.pdf', 'timimoun': 'TIMIMOUN_TRAN_23.pdf', 'elbayadh': 'ELBAYADH_TRAN_23.pdf', 'adrar': 'ADRAR_TRAN_23.pdf', 'souk_ahras': 'SOUK_AHRAS_TRAN_23.pdf', 'elmeniaa': 'ELMENIAA_TRAN_23.pdf', 'blida': 'BLIDA_TRAN_23.pdf', 'alger': 'ALGER_TRAN_23.pdf', 'djelfa': 'DJELFA_TRAN_23.pdf', 'mila': 'MILA_TRAN_23.pdf', 'touggourt': 'TOUGGOURT_TRAN_23.pdf', 'laghouat': 'LAGHOUAT_TRAN_23.pdf', 'boumerdes': 'BOUMERDES_TRAN_23.pdf', 'ghardaia': 'GHARDAIA_TRAN_23.pdf', 'djanet': 'DJANET_TRAN_23.pdf', 'jijel': 'JIJEL_TRAN_23.pdf', 'tipaza': 'TIPAZA_TRAN_23.pdf', 'annaba': 'ANNABA_TRAN_23.pdf', 'bouira': 'BOUIRA_TRAN_23.pdf', 'oum_elbouaghi': 'OUM_ELBOUAGHI_TRAN_23.pdf', 'eltarf': 'ELTARF_TRAN_23.pdf', 'khenchela': 'KHENCHELA_TRAN_23.pdf', 'naama': 'NAAMA_TRAN_23.pdf', 'relizane': 'RELIZANE_TRAN_23.pdf', 'tindouf': 'TINDOUF_TRAN_23.pdf', 'tissemsilt': 'TISSEMSILT_TRAN_

KeyboardInterrupt: 

In [ ]:
# add to vent apartement salle
impo_df = impo_df.append(pd.Series(["mostaganem", "Oued El Kheir",50000,57500,40000,46000,20000,23000], index=impo_df.columns), ignore_index=True)
impo_df = impo_df.append(pd.Series(["chlef", "chlef",25000,28750,20000,23000,16000,18400], index=impo_df.columns), ignore_index=True)
impo_df = impo_df.append(pd.Series(["jijel", "Sidi Maarouf",27600.0,32500.0,25500.0,30000.0,16000.0,19000.0], index=impo_df.columns), ignore_index=True)
impo_df = impo_df.append(pd.Series(["medea", "Berrouaghia",57520,66148,45210,52000,36900,42435], index=impo_df.columns), ignore_index=True)

In [ ]:
# apply to rent apartments
impo_df.loc[impo_df['stt_name'] == 'mascara', ['centre_min', 'centre_max', 'peripherique_min', 'peripherique_max', 'far_min', 'far_max']] /= 10
impo_df.loc[impo_df['stt_name'] == 'tinduf', ['centre_min', 'centre_max', 'peripherique_min', 'peripherique_max', 'far_min', 'far_max']] /= 100
impo_df = impo_df.append(pd.Series(["tizi_ouzou", "Draa Ben Khedda",250,288,80,92,60,69], index=impo_df.columns), ignore_index=True)
impo_df = impo_df.append(pd.Series(["tissemsilt", "Sidi Abed",24,34,20,27,14,20], index=impo_df.columns), ignore_index=True)
impo_df = impo_df.append(pd.Series(["ain defla", "ROUINA",50,58,30,35,30,35], index=impo_df.columns), ignore_index=True)

## For lands

In [ ]:
new_index_names = ['Economique', 'Trerrain_2', 'Trerrain_1','Trerrain_2_sans_raccordements', 'Trerrain_1_sans_raccordements',""]

In [16]:
def clean_table_v5(df):
    """Enhanced function to clean and process the table with improved commune name extraction."""
    name, df =  extract_commune_name_v21(df)
    try:
        # Step 1: Commune name is already provided
        
        # Step 2: Adjusting header, dropping unnecessary columns
        df_cleaned = df.iloc[4:, :].reset_index(drop=True)
        
        # Step 3: Assign new column names
        new_column_names = [
            "type_1", "type_2", "residential_min", "residential_max", 
            "centre_min", "centre_max", "peripherique_min", "peripherique_max","eloigner_min", "eloigner_max"
        ]
        
        if len(df_cleaned.columns) != len(new_column_names):
            print(f"Warning: Skipping table {name} due to unexpected column count.")
            return None

        df_cleaned.columns = new_column_names
        
        # Step 4: Further cleaning
        df_cleaned['type_2'].iloc[3] = 'terrain_2_facade'
        df_cleaned['type_2'].iloc[4] = 'terrain_1_facade'
        
        # Step 5: Numeric Conversion
        numeric_columns = [
            "residential_min", "residential_max", 
            "centre_min", "centre_max", 
            "peripherique_min", "peripherique_max",
            "eloigner_min", "eloigner_max"
        ]
        
        for col in numeric_columns:
            df_cleaned[col] = df_cleaned[col].apply(
                    lambda x: float(x.replace(" ", "").replace("*", "").replace("-", "nan").replace("#REF!", "nan").replace('esavecraccordements','nan').replace('le','nan').replace('cordement','nan').replace('690\n552\n414\n276\n221\n124\n109', '690').replace('avecracnans', 'nan').replace('essansracnans', 'nan').replace('sansracnans', 'nan').replace("nts", "").replace("s", "").replace("t", "")) if isinstance(x, str) and x.strip() not in ["", "/"] else None
            )
        
        # Step 6: Filtering Rows
        df_cleaned = df_cleaned.iloc[[2, 13, 14,15,16,17,18], :]
        
        # Step 7: Setting 'type_2' as the index
        df_cleaned.set_index('type_2', inplace=True)

        # rename the indexies 
        new_index_names = ['Economique', 'Trerrain_2', 'Trerrain_1','Trerrain_2_sans_raccordements', 'Trerrain_1_sans_raccordements',"Indivision","Industriel"]
        df_cleaned.index = new_index_names

        # drop columns 
        df_cleaned.drop(columns=['type_1'], inplace = True)
        
        return name, df_cleaned
        
    except Exception as e:
        print(f'There is an error: {e}')
        return None

In [17]:
from pdfminer.pdfparser import PDFSyntaxError
import pandas as pd



pdfs_path = 'pdf/'

all_data = []

# Get the names of all the pdfs in dict format 
pdfs_in_folder = extract_names_from_pdfs(pdfs_path)

print(pdfs_in_folder)

for key, value in pdfs_in_folder.items():

    print(key)
    # Read the PDFs 
    try:
        dfs = extract_tables_from_pdf(pdfs_path + value)
    except PDFSyntaxError:
        print(f"Warning: The file {value} seems to be an invalid PDF. Skipping...")
        continue

    # Create a list of cleaned dataframes with their respective names
    cleaned_dfs = [clean_table_v5(d) for d in dfs]

    for commune in cleaned_dfs:
        try:
            # Calculate the means
            commune[1]['residential_mean'] = (commune[1]['residential_min'] + commune[1]['residential_max']) / 2
            commune[1]['centre_mean'] = (commune[1]['centre_min'] + commune[1]['centre_max']) / 2
            commune[1]['peripherique_mean'] = (commune[1]['peripherique_min'] + commune[1]['peripherique_max']) / 2
            commune[1]['eloigner_mean'] = (commune[1]['eloigner_min'] + commune[1]['eloigner_max']) / 2

            # Drop the original min and max columns
            commune[1].drop(['residential_min', 'residential_max', 'centre_min', 'centre_max', 
                             'peripherique_min', 'peripherique_max', 'eloigner_min', 'eloigner_max'], axis=1, inplace=True)
            
            # Calculate the ratio as per previous example
            trerrain_1_values = commune[1].loc['Trerrain_1', :]
            ratio = commune[1].div(trerrain_1_values)
            
            # Drop the rows "Economique" and "Trerrain_1" from ratio
            ratio.drop(['Economique', 'Trerrain_1'], inplace=True)
            
            # Flatten the ratio DataFrame and store in all_data
            for index in ratio.index:
                all_data.append({
                    'stt_name': key,
                    'commune_name': commune[0],
                    'type': index,
                    'residential_mean': ratio.loc[index, 'residential_mean'],
                    'centre_mean': ratio.loc[index, 'centre_mean'],
                    'peripherique_mean': ratio.loc[index, 'peripherique_mean'],
                    'eloigner_mean': ratio.loc[index, 'eloigner_mean']
                })
                
        except Exception as e:
            print(f'The error is in wilaya {key}, commune {commune[0]}: {e}')

# Creating a DataFrame from all_data
final_df = pd.DataFrame(all_data)

{'ouargla': 'OUARGLA_TRAN_23.pdf', 'ain_defla': 'AIN_DEFLA_TRAN_23.pdf', 'saida': 'SAIDA_TRAN_23.pdf', 'ouled_djellal': 'OULED_DJELLAL_TRAN_23.pdf', 'timimoun': 'TIMIMOUN_TRAN_23.pdf', 'elbayadh': 'ELBAYADH_TRAN_23.pdf', 'adrar': 'ADRAR_TRAN_23.pdf', 'souk_ahras': 'SOUK_AHRAS_TRAN_23.pdf', 'elmeniaa': 'ELMENIAA_TRAN_23.pdf', 'blida': 'BLIDA_TRAN_23.pdf', 'alger': 'ALGER_TRAN_23.pdf', 'djelfa': 'DJELFA_TRAN_23.pdf', 'mila': 'MILA_TRAN_23.pdf', 'touggourt': 'TOUGGOURT_TRAN_23.pdf', 'laghouat': 'LAGHOUAT_TRAN_23.pdf', 'boumerdes': 'BOUMERDES_TRAN_23.pdf', 'ghardaia': 'GHARDAIA_TRAN_23.pdf', 'djanet': 'DJANET_TRAN_23.pdf', 'jijel': 'JIJEL_TRAN_23.pdf', 'tipaza': 'TIPAZA_TRAN_23.pdf', 'annaba': 'ANNABA_TRAN_23.pdf', 'bouira': 'BOUIRA_TRAN_23.pdf', 'oum_elbouaghi': 'OUM_ELBOUAGHI_TRAN_23.pdf', 'eltarf': 'ELTARF_TRAN_23.pdf', 'khenchela': 'KHENCHELA_TRAN_23.pdf', 'naama': 'NAAMA_TRAN_23.pdf', 'relizane': 'RELIZANE_TRAN_23.pdf', 'tindouf': 'TINDOUF_TRAN_23.pdf', 'tissemsilt': 'TISSEMSILT_TRAN_

### calculate the average of prices 

In [ ]:
final_df['avg_ratio'] = final_df[['centre_mean', 'peripherique_mean', 'eloigner_mean']].mean(axis=1)

In [ ]:
# Function to remove outliers for each land type

def remove_outliers(df, column='avg_ratio'):
    # Empty DataFrame to store results
    df_no_outliers = pd.DataFrame()

    # Loop through each type
    for _, group in df.groupby('type'):
        # Compute Q1 (25th percentile) and Q3 (75th percentile) of the column
        Q1 = group[column].quantile(0.25)
        Q3 = group[column].quantile(0.75)
        IQR = Q3 - Q1

        # Filter out the outliers
        mask = group[column].between(Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)
        df_no_outliers = pd.concat([df_no_outliers, group[mask]])

    return df_no_outliers

# Remove outliers from the dataset
land_ratio_no_outliers = remove_outliers(final_df)

# Display the first few rows of the dataframe without outliers
land_ratio_no_outliers.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Descriptive Statistics for each land type
descriptive_stats = land_ratio_no_outliers.groupby('type')['avg_ratio'].describe()

# Visual Analysis: Box Plot for each land type
plt.figure(figsize=(12, 8))
sns.boxplot(x='type', y='avg_ratio', data=land_ratio_no_outliers)
plt.xticks(rotation=45)
plt.title('Variation of Average Ratios for Each Land Type Across Communes')
plt.xlabel('Land Type')
plt.ylabel('Average Ratio')
plt.show()

descriptive_stats

In [ ]:
land_ratio_no_outliers.groupby('type')['avg_ratio'].quantile(0.5).round(2)

### names corrections for state, commune. for Lands, apartments

In [ ]:
mask = (impo_df['commune_name']=='Unknown')|(impo_df['commune_name']=='Commune de')|(impo_df['commune_name']=="Type d’Immeuble /(Caractéristiques)")| (impo_df['commune_name'].str.len() < 3)

impo_df = impo_df[~mask]
impo_df['commune_name']=impo_df['commune_name'].str.replace(":","", case=False)
impo_df['commune_name']=impo_df['commune_name'].str.replace(".","", case=False)
impo_df['commune_name']=impo_df['commune_name'].str.replace("-","", case=False)
impo_df['commune_name']=impo_df['commune_name'].str.replace("\n","", case=False)

# Replace 'BBA' with 'YES' in the 'STT_NAME' column

impo_df['stt_name'] = impo_df['stt_name'].str.replace('bba', 'Bordj Bou Arreridj',case=False)
impo_df['stt_name'] = impo_df['stt_name'].str.replace('alger', 'algiers',case=False)
impo_df['stt_name'] = impo_df['stt_name'].str.replace('aindefla', 'ain defla',case=False)
impo_df['stt_name'] = impo_df['stt_name'].str.replace('tindouf', 'tinduf',case=False)
#impo_df['commune_name'] = impo_df['commune_name'].replace('Alger Centre', 'Central Algiers')
impo_df['commune_name']=impo_df['commune_name'].str.replace(":","", case=False)
impo_df['commune_name'] = impo_df['commune_name'].str.replace('Bologhine', 'Bulughin',case=False)
impo_df['commune_name'] = impo_df['commune_name'].str.replace('tindouf', 'tinduf',case=False)
impo_df['stt_name'] = impo_df['stt_name'].str.replace('_', ' ')
impo_df['commune_name']=impo_df['commune_name'].str.replace('commune',"", case=False)
impo_df['commune_name']=impo_df['commune_name'].str.replace("d’","", case=False)
impo_df['commune_name']=impo_df['commune_name'].str.replace("é","e", case=False)
impo_df['commune_name'] = impo_df['commune_name'].str.replace('è', 'e',case=False)
impo_df['stt_name'] = impo_df['stt_name'].str.replace('è', 'e',case=False)
impo_df['commune_name']= impo_df['commune_name'].str.replace('Gue de constantine',"Djasr Kasentina", case=False)
impo_df['commune_name']= impo_df['commune_name'].str.replace('Ain naadja',"Djasr Kasentina", case=False)
impo_df['commune_name']=  impo_df['commune_name'].str.replace("Oran Partie","oran", case=False)
impo_df['commune_name']=impo_df['commune_name'].str.replace('central algiers',"Alger centre", case=False)
# Convert far_min and far_max columns to a standard numeric format
#impo_df['far_min'] = impo_df['far_min'].str.replace(' ', '').astype(float)
#impo_df['far_max'] = impo_df['far_max'].str.replace(' ', '').astype(float)
impo_df['commune_name']= impo_df['commune_name'].str.replace('gué de constantine',"Djasr Kasentina", case=False)
impo_df['commune_name']= impo_df['commune_name'].str.replace('Ain naadja',"Djasr Kasentina", case=False)
impo_df['commune_name']=  impo_df['commune_name'].str.replace("Oran Partie","oran", case=False)
impo_df['commune_name']=impo_df['commune_name'].str.replace("djemaa","djemaa beni habibi", case=False)
impo_df['stt_name']=impo_df['stt_name'].str.replace("timouchent","ain temouchent", case=False)
impo_df['commune_name']=impo_df['commune_name'].str.replace("oulhaca","oulhaca el gheraba", case=False)
impo_df['commune_name']=impo_df['commune_name'].str.replace("khezaras","kheraza", case=False)
impo_df['stt_name']=impo_df['stt_name'].str.replace("bbmokhtar","Bordj Badji Mokhtar", case=False)
impo_df['commune_name'] = impo_df['commune_name'].str.replace('EL KENNAR', 'El Kennar Nouchfi',case=False)
impo_df['commune_name'] = impo_df['commune_name'].str.replace('BENYAHIA', 'Benyahia Abderrahmane',case=False)
impo_df['commune_name'] = impo_df['commune_name'].str.replace('è', 'e',case=False)
impo_df['stt_name'] = impo_df['stt_name'].str.replace('è', 'e',case=False)
impo_df['stt_name'] = impo_df['stt_name'].str.replace('Tamenghasset', 'Tamenrasset',case=False)
impo_df['commune_name'] = impo_df['commune_name'].str.replace('Tamenghasset', 'Tamanrasset',case=False)
impo_df['stt_name'] = impo_df['stt_name'].str.replace('inguezzam', 'In Guezzam',case=False)

impo_df.reset_index(drop=True,inplace=True)

# Read the geojson (wilaya, commune) and clean it (to get from 'https://osm-boundaries.com/Map')

In [ ]:
import geopandas as gpd

# Read GeoJSON file
gdf = gpd.read_file('algeria.geojson')

# drop the rows where admin_level	== 6
gdf = gdf[(gdf.admin_level	== 4) |(gdf.admin_level	== 8)]
gdf.drop(columns=['boundary','local_name',	'name_en'], inplace=True)
gdf.reset_index(drop=True,inplace=True)

# drop the df to 2.  
df_admin_4 = gdf[gdf['admin_level'] == 4]
df_admin_8 = gdf[gdf['admin_level'] == 8]

# Function to extract the wilaya name for each row in df_admin_8
def get_wilaya_name(row):
    # Split the parents values and get the middle value (osm_id of wilaya)
    wilaya_osm_id = int(row['parents'].split(',')[1])
    
    # Find the name from df_admin_4 where osm_id matches the extracted osm_id
    wilaya_name = df_admin_4[df_admin_4['osm_id'] == wilaya_osm_id]['name'].iloc[0]
    
    return wilaya_name

# Apply the function to df_admin_8 to create the new 'wilaya' column
df_admin_8['wilaya'] = df_admin_8.apply(get_wilaya_name, axis=1)
df_admin_8 = df_admin_8[['wilaya','name','geometry']]

# drop the not used df
del df_admin_4
del gdf

###  Get all the villages and communes in the states

In [ ]:
ox.settings.overpass_settings = '[out:json][timeout:180]'
ox.settings.use_cache = True
# Revised code using the existing commune_geometry

all_data_revised = []

for _, row in df_admin_8.iterrows():
    # Extract stt_name, commune_name, and commune_geometry
    stt_name = row['wilaya']
    commune_name = row['name']
    commune_geometry = row['geometry']
    
    try:
        # Get places within the commune's boundary
        places = ox.features_from_polygon(commune_geometry, tags={'place': ['village', 'city']})
        
        for _, place_row in places.iterrows():
            place_name = place_row.get('name', 'Unknown')
            place_type = place_row.get('place', 'Unknown')
            place_point = place_row['geometry'].centroid
            all_data_revised.append({
                        'stt_name': stt_name,
                        'commune_name': commune_name,
                        'place_name': place_name,
                        'place_type': place_type,  # This will be either 'village' or 'city'
                        'latitude': place_point.y,
                        'longitude': place_point.x
                    })

    except Exception as e:
        print(f"Error processing commune {commune_name} in wilaya {stt_name}: {e}")
    
    time.sleep(2)

# Display the first few entries in all_data_revised
df_places = pd.DataFrame(all_data_revised)

### Mergethe two df

In [ ]:
# Assuming df1 and df2 are based on the previously loaded and processed dataframes

# Merging df1 and df2 on the specified columns to get the area_km2 value for each commune in df1
df1_merged = pd.merge(df_places, df_admin_8[['wilaya', 'name','geometry']], 
                      left_on=['stt_name', 'commune_name'], 
                      right_on=['wilaya', 'name'], 
                      how='left')

# Renaming the area_km2 column to commune_area
#df1_merged.rename(columns={'area_km2': 'commune_area'}, inplace=True)

# Dropping the redundant columns from the merge
df1_with_area = df1_merged.drop(columns=['wilaya', 'name'])

# Displaying the first few rows of the resultant dataframe
df1_with_area.head()

### Getting the center of the commune that didnt has tags

In [ ]:
import osmnx as ox

# OSMnx settings
#ox.config(use_cache=True, log_console=True, user_agent="my_geocode_app")

# Function to get latitude and longitude for a wilaya and commune combination
def get_lat_lon(row):
    try:
        location = ox.geocode(f"{row['name']}, {row['wilaya']}, Algeria")
        return location
    except:
        return (None, None)

# Assuming your dataframe is named df
df_admin_8['latitude_center'], df_admin_8['longitude_center'] = zip(*df_admin_8.apply(get_lat_lon, axis=1))

In [ ]:
df_admin_8_copy =df_admin_8.copy()
df_admin_8_copy['place_name'] = df_admin_8_copy['name']
df_admin_8_copy['place_type'] = 'city'

In [ ]:
# Reorganize the columns
df_admin_8_copy = df_admin_8_copy[['wilaya', 'name', 'place_name', 'place_type', 'latitude_center', 'longitude_center', 'geometry']]

# Rename the columns
df_admin_8_copy = df_admin_8_copy.rename(columns={
    'wilaya': 'stt_name',
    'name': 'commune_name',
    'latitude_center': 'latitude',
    'longitude_center': 'longitude',
    'area_km2': 'commune_area'
})

# Now 'df' contains the DataFrame with the desired column order and renamed columns

# concat the 2 df 
final_villages_df = pd.concat([df1_with_area,df_admin_8_copy],axis=0)

#drop the duplicated rows

final_villages_df.drop_duplicates(subset=['commune_name' ,'place_name']) 

final_villages_df.reset_index(drop=True, inplace=True)

final_villages_df

### Creating new columns centre, perepheric

In [ ]:
final_villages_df.reset_index(drop=True, inplace=True)

In [ ]:
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("Al Muhammadiyya",'Mohammadia', case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("Qudayyil",'Gdyel', case=False)
final_villages_df['stt_name'] = final_villages_df['stt_name'].str.replace("Médéa",'Medea', case=False)
final_villages_df['stt_name'] = final_villages_df['stt_name'].str.replace("é","e", case=False)
final_villages_df['stt_name'] = final_villages_df['stt_name'].str.replace("è","e", case=False)
final_villages_df['stt_name'] = final_villages_df['stt_name'].str.replace("ï","i", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("maalma",'mehalma', case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("é","e", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("ï","i", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace('central algiers',"Alger centre", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("ash shalif","chlef", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("é","e", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("è","e", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("ï","i", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("ash shalif","chlef", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("bashar","bechar", case=False)
final_villages_df['stt_name'] = final_villages_df['stt_name'].str.replace("bashar","bechar", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("tulqa","tolga", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("burj al kifan","Bordj El Kiffan", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("herraoua","Heraoua", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("bab al wadi","Bab El Oued", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("bachdjarah","Bachdjerrah", case=False)
final_villages_df['commune_name'] = final_villages_df['commune_name'].str.replace("aghbal ⴰⵖⴱⴰⵍ أغبال","aghbal", case=False)

In [ ]:
# Convert the strings to shapely geometries (if you load it)
from shapely.wkt import loads

def convert_to_geometry(data):
    try:
        return loads(data)
    except:
        return data

# Convert the strings to shapely geometries
final_villages_df['geometry'] = final_villages_df['geometry'].apply(convert_to_geometry)

## Create geometries cercales for center, perepherique, far

In [ ]:
# Define the necessary functions as provided

def point_to_3857(point):
    """Convert a point from EPSG:4326 to EPSG:3857"""
    gseries = gpd.GeoSeries([point], crs="EPSG:4326")
    return gseries.to_crs(epsg=3857).iloc[0]

def create_center_geometry(row):
    # Convert latitude and longitude to a Point in EPSG:3857
    point = Point(row['longitude'], row['latitude'])
    point_in_meters = point_to_3857(point)
    
    # Create buffer based on place_type
    if row['place_type'] == 'city':
        return point_in_meters.buffer(500)
    elif row['place_type'] == 'village':
        return point_in_meters.buffer(200)

def create_peripherique_geometry(row):
    # Convert latitude and longitude to a Point in EPSG:3857
    point = Point(row['longitude'], row['latitude'])
    point_in_meters = point_to_3857(point)
    
    # Create buffer based on place_type
    if row['place_type'] == 'city':
        return point_in_meters.buffer(1000)
    elif row['place_type'] == 'village':
        return point_in_meters.buffer(400)

def intersect_with_geometry(row, column_name):
    """Intersect a given geometry column with the main geometry."""
    return row[column_name].intersection(row['geometry'])

def subtract_center_from_peripherique(row):
    """Subtract the center geometry from the peripherique geometry."""
    return row['peripherique'].difference(row['center'])

# Now apply the transformations to the dataframe

# Convert the DataFrame to a GeoDataFrame using the 'center' column
gdf_center = gpd.GeoDataFrame(final_villages_df, geometry='geometry')
gdf_center.crs = "EPSG:4326"
final_villages_df['geometry'] = gdf_center['geometry'].to_crs("EPSG:3857")

# Create center and peripherique geometries
final_villages_df['center'] = final_villages_df.apply(create_center_geometry, axis=1)
final_villages_df['peripherique'] = final_villages_df.apply(create_peripherique_geometry, axis=1)


### at this point ['center','peripherique'] are  in "EPSG:3857" and [geometry] is  "EPSG:4326". you should converand them to 'EPSG:4326'   ###


# Intersect the center and peripherique geometries with the main geometry
final_villages_df['center'] = final_villages_df.apply(lambda row: intersect_with_geometry(row, 'center'), axis=1)
final_villages_df['peripherique'] = final_villages_df.apply(lambda row: intersect_with_geometry(row, 'peripherique'), axis=1)

# Subtract the center geometry from the peripherique geometry
final_villages_df['peripherique'] = final_villages_df.apply(subtract_center_from_peripherique, axis=1)

# transform the columns back to  'EPSG:4326'

final_villages_df['geometry'] = gpd.GeoSeries(final_villages_df['geometry'], crs="EPSG:3857").to_crs(epsg=4326)
final_villages_df['center'] = gpd.GeoSeries(final_villages_df['center'], crs="EPSG:3857").to_crs(epsg=4326)
final_villages_df['peripherique'] = gpd.GeoSeries(final_villages_df['peripherique'], crs="EPSG:3857").to_crs(epsg=4326)
final_villages_df.head()

### Merge impo_df with final_villages_df

In [ ]:
from fuzzywuzzy import fuzz

# 1. Precompute the choices
choices = impo_df.apply(lambda x: (x['stt_name'] + " " + x['commune_name']).lower(), axis=1).tolist()

# 2. Vectorized function to get best matches using fuzz.ratio
def get_best_match_vectorized(combined_name):
    scores = [fuzz.ratio(combined_name, choice) for choice in choices]
    max_score_index = scores.index(max(scores))
    
    # If the score is above a certain threshold, return the index
    if scores[max_score_index] > 80:
        return max_score_index
    return None

# Applying the function to get best matches
combined_names_series = (final_villages_df['stt_name'] + " " + final_villages_df['commune_name']).str.lower()
final_villages_df['matched_index'] = combined_names_series.apply(get_best_match_vectorized)

# 3. Merge the two DataFrames on the new index
final_villages_df = pd.merge(final_villages_df, impo_df, left_on="matched_index", right_index=True, how="left", suffixes=('', '_y'))

# Keeping only the relevant columns and dropping duplicated columns
columns_to_keep = ['stt_name', 'commune_name', 'place_name', 'place_type', 'latitude', 'longitude', 'geometry', 'center', 'peripherique', 
                   'centre_min', 'centre_max', 'peripherique_min', 'peripherique_max', 'far_min', 'far_max']
final_villages_df = final_villages_df[columns_to_keep]

final_villages_df.head()

In [ ]:
# Filter rows where stt_name is "algiers" and place_type is "city"
algiers_city_df = final_villages_df[(final_villages_df['stt_name'] == 'algiers') & (final_villages_df['place_type'] == 'city')]

# Keep all rows from other stt_name values without any filtering
other_stt_df = final_villages_df[final_villages_df['stt_name'] != 'algiers']

# Concatenate the two filtered dataframes
final_villages_df = pd.concat([algiers_city_df, other_stt_df])

# Drop the 'place_name' and 'place_type' columns
final_villages_df = final_villages_df.drop(columns=['place_name', 'place_type'])

# Identify repeated communes within the same stt_name
repeated_communes = final_villages_df.groupby(['stt_name', 'commune_name']).size().reset_index(name='counts')
repeated_communes = repeated_communes[repeated_communes['counts'] > 1]

### Put all the geometries of the same state in one geometry

In [ ]:
from shapely import wkt
from shapely.ops import unary_union

# Function to combine geometries
def combine_geometries(df, column_name):
    # Convert the WKT strings to shapely geometries
    geometries = df[column_name]
    
    # Clean and filter out invalid geometries
    cleaned_geometries = [geom.buffer(0) for geom in geometries if geom.is_valid]
    
    # Combine the geometries
    combined_geometry = unary_union(cleaned_geometries)
    return combined_geometry  # Return the shapely geometry

# Iterate over the repeated communes and combine their geometries
for _, row in repeated_communes.iterrows():
    mask = (final_villages_df['stt_name'] == row['stt_name']) & (final_villages_df['commune_name'] == row['commune_name'])
    subset = final_villages_df[mask]
    
    final_villages_df.loc[mask, 'center'] = combine_geometries(subset, 'center')
    final_villages_df.loc[mask, 'peripherique'] = combine_geometries(subset, 'peripherique')

# After combining, we can keep only the first row of the repeated communes and drop the rest
for _, row in repeated_communes.iterrows():
    mask = (final_villages_df['stt_name'] == row['stt_name']) & (final_villages_df['commune_name'] == row['commune_name'])
    indices_to_drop = final_villages_df[mask].index[1:]
    final_villages_df.drop(indices_to_drop, inplace=True)

final_villages_df.reset_index(drop=True, inplace=True)
final_villages_df